# parcels regions

runs parcels on existing netcdf files

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from pyplume.constants import CONFIGS_DIR
import pyplume.utils as utils
from pyplume.config_parser import prep_sim_from_cfg, process_results

## configuration

change the contents of `configs` for the simulation configuration you want

In [ ]:
configs = [
    # "plume_track.json",
    # "plume_track_alongshore.json",
    "plume_track_live.json",
    # "mwb.json",
    # "tijuana_now.json",
#     "buoy_track_704_02.json",
    # "hunington_all.json",
#     "dummy_sim.json",
]

In [ ]:
loaded_configs = [utils.load_config(CONFIGS_DIR / path) for path in configs]
sims = []
for cfg in loaded_configs:
    sims.append(prep_sim_from_cfg(cfg))

## Animated gif stuff and particle simulation

In [ ]:
[sim.pset.show(field="vector") for sim in sims]

### simulation execution

In [ ]:
for sim in sims:
    sim.execute()

### plot and gif generation

gif generation requires [magick](https://imagemagick.org/index.php)

if it doesn't work just comment out the gif generation line

In [ ]:
for cfg, sim in zip(loaded_configs, sims):
    process_results(sim, cfg)